In [ ]:
#using Rebugger
include("jlboosttree.jl")
include("jlboost.jl")

using DataFrames, RCall, StatsBase, CSV,ForwardDiff
using Plots
gr()

# set up global parameters
lambda = 1
gamma  = 3

@time df = CSV.read(joinpath(pwd(),"data","cs-training.csv"))
rename!(df,Symbol("NumberOfTime30-59DaysPastDueNotWorse") => :NumberOfTime30_59DaysPastDueNotWorse,Symbol("NumberOfTime60-89DaysPastDueNotWorse") => :NumberOfTime60_89DaysPastDueNotWorse);

na2n(x) = x == "NA" ? Int32(0) : parse(Int32, x)
na2n(x::Integer) = x
df[:NumberOfDependents] = na2n.(df[:NumberOfDependents]);
df[:MonthlyIncome] = na2n.(df[:MonthlyIncome]);

[typeof(c[2]) for c in eachcol(df)]


# setup the initial weights
df[:prev_w] = 0.0;

# set a feature to split on
target = :SeriousDlqin2yrs
prev_w = :prev_w

features = names(df)[3:end-1][1:2]#[[1:4;6:9]]
#features = [:RevolvingUtilizationOfUnsecuredLines,:NumberRealEstateLoansOrLines]
#features = [:NumberRealEstateLoansOrLines]
#features = [:MonthlyIncome]
#features = features[10:10]

# function oklah(df, target, feature1; lambda = lambda, gamma = gamma, maxdepth = maxdepth)
#      booster1 = xgboost(df, target, feature1; lambda = lambda, gamma = gamma, maxdepth = maxdepth)
#      scoretree(df, booster1, :new_weight);
#      (gini2, cudata) = gini(-df[:new_weight], df[target]; plotauc = true);
#      gini2
# end
#
# @time [oklah(df, target, [feature1]; lambda = lambda, gamma = gamma, maxdepth = 2) for feature1 in features]
#
# @time oklah(df, target, features[1:1]; lambda = lambda, gamma = gamma)
# @code_warntype oklah(df, target, features[1:1]; lambda = lambda, gamma = gamma)

@time booster1 = xgboost(df, target, features; lambda = lambda, gamma = gamma)
delete!(df,:new_weight)
df[:new_weight] = 0.0
@time scoretree(df, booster1, :new_weight);
@time (gini2, cudata) = gini(-df[:new_weight], df[target]; plotauc = true);
gini2

In [ ]:
df1 = by(df, :new_weight, df1->DataFrame(d=sum(df1[target]), n=size(df1)[1]))
df1[:dr] = df1[:d]./df1[:n];
df1[:pd] = softmax.(df1[:new_weight])
sort(df1,:new_weight)

sort(by(df,:new_weight, df1->(minimum(df1[:age]),maximum(df1[:age]))), :new_weight)

plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])

@time booster2 = xgboost(df, target, features; prev_w = :new_weight, lambda = lambda, gamma = gamma)
@time scoretree(df, booster2, :new_weight2);
(gini2, cudata) = gini(-(df[:new_weight2].+df[:new_weight]), df[target]; plotauc = true);
gini2
plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])
df[:new_weight_all] = df[:new_weight2] .+ df[:new_weight]

@time booster3 = xgboost(df, target, features; prev_w = :new_weight_all, lambda = 1, gamma = gamma)
@time scoretree(df, booster3, :new_weight3);
(gini3, cudata) = gini(-(df[:new_weight3].+df[:new_weight_all]), df[target]; plotauc = true);
gini3
plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])

@time booster4 = xgboost(df, target, features; prev_w = :new_weight_all, lambda = 1, gamma = gamma)
@time scoretree(df, booster4, :new_weight4);
(gini4, cudata) = gini(-(df[:new_weight4].+df[:new_weight_all]), df[target]; plotauc = true);
print(gini4)
plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])

df[:new_weight_all] .= df[:new_weight_all] .+ df[:new_weight4]
@time booster5 = xgboost(df, target, features; prev_w = :new_weight_all, lambda = 1, gamma = gamma)
@time scoretree(df, booster5, :new_weight5);
(gini4, cudata) = gini(-(df[:new_weight5].+df[:new_weight_all]), df[target]; plotauc = true);
print(gini4)
plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])

plot([gini2, gini3, gini4])

In [32]:
using DataFrames, RCall, ForwardDiff, StatsBase, CSV
using Plots
gr()

Plots.GRBackend()

In [33]:
include(joinpath(pwd(),"JLBoostTree.jl"))
include(joinpath(pwd(),"jlboost.jl"))

gini (generic function with 1 method)

In [34]:
R"""
iris1 = iris
"""
@rget iris1
iris1;
unique(iris1[:Species])

3-element Array{String,1}:
 "setosa"    
 "versicolor"
 "virginica" 

In [35]:
df = iris1;
df[:issetosa] = t2one.(df[:Species] .== "virginica");

In [36]:
# set up global parameters
lambda = 1
gamma  = 3

3

In [37]:
@time df = CSV.read(joinpath(pwd(),"data","cs-training.csv"))
rename!(df,  
    Symbol("NumberOfTime30-59DaysPastDueNotWorse") => :NumberOfTime30_59DaysPastDueNotWorse,
    Symbol("NumberOfTime60-89DaysPastDueNotWorse") => :NumberOfTime60_89DaysPastDueNotWorse
);

  0.601172 seconds (4.80 M allocations: 99.486 MiB, 4.37% gc time)


In [20]:
#countmap(df[:SeriousDlqin2yrs])
#([countmap(ismissing.(c[2])) for c in eachcol(df)], [typeof(c[2]) for c in eachcol(df)])

In [38]:
#countmap(df[:NumberOfDependents])
na2n(x) = x == "NA" ? 0 : parse(Int8, x)
df[:NumberOfDependents] = na2n.(df[:NumberOfDependents]);

In [39]:
# setup the initial weights
df[:prev_w] = 0.0
df;

In [40]:
# set a feature to split on
target = :issetosa
feature = :Petal_Length
prev_w = :prev_w

features = [:Sepal_Length, :Sepal_Width, :Petal_Length, :Petal_Width]

4-element Array{Symbol,1}:
 :Sepal_Length
 :Sepal_Width 
 :Petal_Length
 :Petal_Width 

In [62]:
# set a feature to split on
target = :SeriousDlqin2yrs
prev_w = :prev_w

features = names(df)[3:end-1][1:10]#[[1:4;6:9]]
#features = [:RevolvingUtilizationOfUnsecuredLines,:NumberRealEstateLoansOrLines]
#features = [:NumberRealEstateLoansOrLines]

10-element Array{Symbol,1}:
 :RevolvingUtilizationOfUnsecuredLines
 :age                                 
 :NumberOfTime30_59DaysPastDueNotWorse
 :DebtRatio                           
 :MonthlyIncome                       
 :NumberOfOpenCreditLinesAndLoans     
 :NumberOfTimes90DaysLate             
 :NumberRealEstateLoansOrLines        
 :NumberOfTime60_89DaysPastDueNotWorse
 :NumberOfDependents                  

In [63]:
function oklah(df, target, feature1; lambda = lambda, gamma = gamma)
     booster1 = xgboost(df, target, feature1; lambda = lambda, gamma = gamma)
     scoretree(df, booster1, :new_weight);
     (gini2, cudata) = gini(-df[:new_weight], df[target]; plotauc = true);
     gini2
end

@time [oklah(df, target, [feature1]; lambda = lambda, gamma = gamma) for feature1 in features]

116.542328 seconds (362.53 M allocations: 23.127 GiB, 16.64% gc time)


10-element Array{Float64,1}:
  0.5360397812354545  
  0.2492784646585542  
  0.35363891415652615 
  0.18346321630427576 
 -0.012080907640135585
  0.13098287585610735 
  0.2932391635082119  
  0.11720025334131279 
  0.22698192632488845 
 -0.3886622441651705  

In [46]:
features = [:MonthlyIncome]

,id,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30_59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60_89DaysPastDueNotWorse,NumberOfDependents,prev_w,new_weight
1,1,1,0.766127,45,2,0.802982,9120,13,0,6,0,2,0.0,-1.78059
2,2,0,0.957151,40,0,0.121876,2600,4,0,0,0,1,0.0,-2.19701
3,3,0,0.65818,38,1,0.0851134,3042,2,1,0,0,0,0.0,-2.19701
4,4,0,0.23381,30,0,0.0360497,3300,5,0,0,0,0,0.0,-2.19701
5,5,0,0.907239,49,1,0.0249257,63588,7,0,1,0,0,0.0,-2.63892
6,6,0,0.213179,74,0,0.375607,3500,3,0,1,0,1,0.0,-2.40477
7,7,0,0.305682,57,0,5710.0,NA,8,0,3,0,0,0.0,-1.78059
8,8,0,0.754464,39,0,0.20994,3500,8,0,0,0,0,0.0,-2.40477
9,9,0,0.116951,27,0,46.0,NA,2,0,0,0,0,0.0,-1.78059
10,10,0,0.189169,57,0,0.606291,23684,9,0,4,0,2,0.0,-2.33029


In [43]:
@time booster1 = xgboost(df, target, features; lambda = lambda, gamma = gamma)

 11.967519 seconds (33.21 M allocations: 1.807 GiB, 9.51% gc time)


 MonthlyIncome, weight = 0.0
   -- MonthlyIncome <= 6643
     ---- MonthlyIncome, weight = -1.7037936913895992
     -- MonthlyIncome <= 13922
       ---- MonthlyIncome, weight = -0.812687652813303
       -- MonthlyIncome <= 10662
         ---- MonthlyIncome, weight = -0.5023096728953607
         -- MonthlyIncome <= 10350
           ---- MonthlyIncome, weight = -0.045799485176623446
           -- MonthlyIncome <= 100
             ---- weight = -0.1846246484770146
           -- MonthlyIncome > 100
             ---- MonthlyIncome, weight = 0.13717916045893921
             -- MonthlyIncome <= 1000
               ---- weight = 0.5000079825999485
             -- MonthlyIncome > 1000
               ---- weight = -0.08056598276351708
         -- MonthlyIncome > 10350
           ---- weight = -0.41813704433272764
       -- MonthlyIncome > 10662
         ---- MonthlyIncome, weight = -0.2983387462339021
         -- MonthlyIncome <= 10668
           ---- weight = 0.9211228764408331
         -- Mon

In [47]:
@time scoretree(df, booster1, :new_weight);
@time (gini2, cudata) = gini(-df[:new_weight], df[target]; plotauc = true);
gini2

  0.282032 seconds (1.12 k allocations: 17.887 MiB, 4.01% gc time)
  0.072555 seconds (1.64 M allocations: 45.711 MiB, 20.47% gc time)


-0.012080907640135585

In [ ]:
plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])

In [61]:
ok=sort(by(df, :new_weight, df1->(sum(df1[target])/size(df1)[1], size(df1)[1])), :new_weight)
ok
#plot(ok[:,1], ok[:,2])
ok[:pd] = softmax.(ok[:new_weight])
ok

,new_weight,x1,pd
1,-3.43693,"(0.0278054, 2014)",0.0311611
2,-3.31498,"(0.0294118, 272)",0.0350609
3,-3.24922,"(0.0366446, 2265)",0.0373551
4,-3.20623,"(0.0275, 400)",0.0389318
5,-3.00798,"(0.0468986, 2644)",0.0470668
6,-2.85736,"(0.0543001, 9558)",0.0543021
7,-2.72687,"(0.057947, 604)",0.0614061
8,-2.72041,"(0.0601504, 1064)",0.0617796
9,-2.63892,"(0.0666463, 16400)",0.0666751
10,-2.57647,"(0.0703829, 5328)",0.070668


In [83]:
countmap(df[:new_weight])

Dict{Float64,Int64} with 26 entries:
  -0.357471 => 759
  -0.983923 => 70
  -0.939205 => 1771
  0.229765  => 14
  -1.42912  => 2686
  0.444767  => 10
  -2.78084  => 16332
  -0.496375 => 3721
  -2.14926  => 397
  -1.17068  => 1022
  -0.733029 => 635
  -1.83103  => 2224
  -2.6425   => 109686
  -1.21848  => 413
  0.789742  => 70
  0.0429011 => 108
  -1.37362  => 107
  -2.16418  => 1343
  -1.80417  => 173
  -2.67784  => 5900
  -1.72619  => 1233
  -1.41227  => 389
  0.0671791 => 48
  -0.724351 => 30
  -0.106245 => 754
  -0.271718 => 105

In [ ]:
@time booster2 = xgboost(df, target, features; prev_w = :new_weight, lambda = lambda, gamma = gamma)

In [ ]:
@time scoretree(df, booster2, :new_weight2);

In [ ]:
(gini2, cudata) = gini(-(df[:new_weight2].+df[:new_weight]), df[target]; plotauc = true);
gini2

In [ ]:
plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])

In [88]:
df[:new_weight_all] = df[:new_weight2] .+ df[:new_weight]
countmap(df[:new_weight_all])

KeyError: KeyError: key :new_weight2 not found

In [ ]:
@time booster3 = xgboost(df, target, features; prev_w = :new_weight_all, lambda = 1, gamma = gamma)
@time scoretree(df, booster3, :new_weight3);

In [ ]:
(gini3, cudata) = gini(-(df[:new_weight3].+df[:new_weight_all]), df[target]; plotauc = true);
gini3

In [ ]:
plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])

In [ ]:
df[:new_weight_all] .= df[:new_weight_all] .+ df[:new_weight3]
countmap(df[:new_weight_all])

In [ ]:
@time booster4 = xgboost(df, target, features; prev_w = :new_weight_all, lambda = 1, gamma = gamma)
@time scoretree(df, booster4, :new_weight4);
(gini4, cudata) = gini(-(df[:new_weight4].+df[:new_weight_all]), df[target]; plotauc = true);
print(gini4)
plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])

In [ ]:
df[:new_weight_all] .= df[:new_weight_all] .+ df[:new_weight4]
countmap(df[:new_weight_all])

In [ ]:
@time booster5 = xgboost(df, target, features; prev_w = :new_weight_all, lambda = 1, gamma = gamma)
@time scoretree(df, booster5, :new_weight5);
(gini4, cudata) = gini(-(df[:new_weight5].+df[:new_weight_all]), df[target]; plotauc = true);
print(gini4)
plot(vcat(0,cudata[1]),vcat(0,cudata[2]))
plot!([0,1],[0,1])